In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
# importing time to make delays
import time

import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display,HTML

C:\Users\mdima\AppData\Local\Temp\ipykernel_17196\3433573634.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


In [5]:
def click(x):
    button = driver.find_element(By.XPATH, x)
    button.click()
    time.sleep(2)

#### how many discounted items you want to check?

In [3]:
get_item = input("how many items on sale would you like to check? ")

#### this part might take long, depending on how many items we want to check

In [7]:
url = "https://www.decathlon.de/special-page/_/N-x6qfruZzB50HB"
# --disabling browser popup notifications--
options = Options()
options.add_argument("--disable-notifications")
# to prevent from being spotted as a robot
options.add_argument('--disable-gpu')
options.add_argument('user-agent=fake-useragent')
# installing chromedriver, so that we dont need to keep the chromedriver file
# that needs to be updated every once in a while. better install the latest automatically
driver = webdriver.Chrome(ChromeDriverManager().install())
# opens the browser, maximize window size
driver.maximize_window()
# opening url
driver.get(url)
# wait a bit until the "I accept" popup shows
time.sleep(2)
accept = "/html/body/div[1]/div/div/div/div/div/div[3]/button[2]"
click(accept)
more = "/html/body/div[6]/div[2]/div[2]/div[4]/div[2]/div/div[1]/button"
stop = False
while stop == False:
    for i in range(2):
        driver.execute_script("window.scrollTo(0, window.scrollY + 1500)") 
        time.sleep(.2)
    try:
        click(more)
    except:
        pass
    # getting page source after scrolling
    html = driver.page_source
    soup = bs4(html)
    items = soup.find_all("article",{"class":"dkt-product js-product-slider-init"})
    if len(items) >= int(get_item):
        stop = True

C:\Users\mdima\AppData\Local\Temp\ipykernel_17196\827299242.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


#### create dataframe to contain the results. run all cells below

In [35]:
def get_pic(path):
    return '<img src=\"'+path+'\" width=\"100\" >'

In [57]:
lprev_price = []
ldisc = []
lnew_price = []
lbrand = []
ldesc = []
lstar = []
lcolor = []
limg = []
for i in items:
    try:
        prev_price = i.find("span",{"class":"dkt-price__previous-price"}).text
        to_replace = ["€","."]
        for j in to_replace:
            prev_price = prev_price.replace(j, "")
        prev_price = prev_price.replace(",",".")
        prev_price = int(prev_price)
    except:
        pass
    try:
        disc = i.find("span",{"class":"dkt-price__reduction"}).text
        to_replace = ["-","%"]
        for j in to_replace:
            disc = disc.replace(j, "")
        disc = int(disc)
    except:
        pass
    try:
        new_price = int(i.find("div",{"class":"dkt-price__cartridge"}).get("data-price"))
    except:
        pass
    try:
        brand = i.find("span",{"itemprop":"name"}).text.strip()
    except:
        pass
    try:
        star = 1*(len(i.find_all("span",{"class":"icon icon-star"})))+(len(i.find_all("span",{"class":"icon icon-star star-half"})))/2
    except:
        pass
    try:
        color = i.find("p",{"class":"dkt-product-slider__thumbnails__colors-infos"}).text
    except:
        pass
    try:
        img = i.find("picture",{"class":"imageclass"}).find("img").get("src")
    except:
        pass
    lprev_price.append(prev_price)
    ldisc.append(disc)
    lnew_price.append(new_price)
    lbrand.append(brand)
    lstar.append(star)
    lcolor.append(color)
    limg.append(img)

In [58]:
df = pd.DataFrame()
df["previous_price"] = lprev_price
df["discount_in_%"] = ldisc
df["reduced_price"] = lnew_price
df["brand"] = lbrand
df["rating"] = lstar
df["colors"] = lcolor
df["photo"] = limg

#### search result(s)

In [59]:
image_cols = ["photo"]
format_dict = {}
format_dict["photo"] = get_pic
display(HTML(df.to_html(escape=False ,formatters=format_dict)))

,previous_price,discount_in_%,reduced_price,brand,rating,colors,photo
0,19.99,70,39,KIPRUN,4.0,2 Farben,
1,419.99,14,39,ROCKRIDER,4.5,2 Farben,
2,69.99,35,39,ROCKRIDER,3.5,2 Farben,
3,14.99,40,39,KIPRUN,4.5,2 Farben,
4,9.99,50,39,DOMYOS,4.5,2 Farben,
5,5.99,16,39,ARTENGO,5.0,6 Farben,
6,17.99,72,39,PUMA,0.0,1 Farben,
7,9.99,40,39,WEDZE,4.5,2 Farben,
8,7.99,25,39,WEDZE,4.5,3 Farben,
9,5.99,33,39,OXELO,4.5,2 Farben,
